<a href="https://colab.research.google.com/github/surmehta1/mgmt467-analytics-portfolio/blob/main/Unit2_Lab2_PromptStudio_Tasks0to4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🤖 MGMT 467 - Unit 2 Lab 2: Prompt Studio for AI-Assisted SQL + ML

**Date:** 2025-10-16  
**Objective:** Build and refine a complete ML pipeline for churn prediction using BigQuery — but with **Gemini-style prompts** guiding SQL generation.

You'll learn to:
- Frame SQL goals as clear prompts
- Generate, test, and debug queries with an AI assistant
- Reflect on each modeling step and your prompt design



## Task 0: Connect to BigQuery

**🎯 Goal:** Verify BigQuery access from Colab.  
**📌 Requirements:** Use `%%bigquery`, get current date and user session.

---

### 🧠 Prompt Template  
> Write a SQL query that returns CURRENT_DATE() and SESSION_USER(). I will run it with %%bigquery in Colab.

---

### 👩‍🏫 Example Prompt  
> Write a SQL query using BigQuery syntax that returns today’s date and the current session user.

---

### ✅ Expected SQL Output
```sql
SELECT CURRENT_DATE() AS today, SESSION_USER() AS user;
```

---

### 🔍 Checkpoint  
Query should return a single row with today's date and your user.



## Task 1: Prepare ML Table

**🎯 Goal:** Create a clean features table for modeling churn.  
**📌 Requirements:** Use cleaned_features as source, select relevant columns, filter rows with churn_label IS NOT NULL.

---

### 🧠 Prompt Template  
> Write a query that creates a new table with columns: [region, plan_tier, age_band, ...] and churn_label from [source_table]. Filter to rows where churn_label IS NOT NULL.

---

### 👩‍🏫 Example Prompt  
> Create a BigQuery table named churn_features from cleaned_features with selected features and where churn_label IS NOT NULL.

---

### ✅ Expected SQL Output
```sql
CREATE OR REPLACE TABLE `your_dataset.churn_features` AS
SELECT region, plan_tier, age_band, avg_rating, total_minutes, churn_label
FROM `your_dataset.cleaned_features`
WHERE churn_label IS NOT NULL;
```

---

### 🔍 Checkpoint  
Table should appear in BigQuery and contain non-null labels.


In [ ]:
from google.colab import auth
auth.authenticate_user()

project_id = "mgmt467-471119"  # replace with your project ID if different
!gcloud config set project $project_id

Updated property [core/project].


In [ ]:
%%bigquery --project $project_id
SELECT CURRENT_DATE() AS today, SESSION_USER() AS user;

Query is running:   0%|          |

Downloading:   0%|          |

,today,user
0,2025-10-25,mehtasur27@gmail.com



## Task 2: Train Logistic Regression Model

**🎯 Goal:** Train a basic BQML logistic regression model.  
**📌 Requirements:** Use churn_features table, predict churn_label from features.

---

### 🧠 Prompt Template  
> Write a CREATE MODEL SQL for logistic regression using churn_label as label and [features] as inputs.

---

### 👩‍🏫 Example Prompt  
> Train a logistic regression model to predict churn_label using region, plan_tier, total_minutes, avg_rating.

---

### ✅ Expected SQL Output
```sql
CREATE OR REPLACE MODEL `your_dataset.churn_model`
OPTIONS(model_type='logistic_reg') AS
SELECT region, plan_tier, total_minutes, avg_rating, churn_label
FROM `your_dataset.churn_features`;
```

---

### 🔍 Checkpoint  
Model appears in BigQuery under Models. Training completes.


In [ ]:
%%bigquery --project $project_id
CREATE OR REPLACE MODEL `mgmt467-471119.netflix.churn_model_final`
OPTIONS(model_type='logistic_reg') AS
SELECT
  region,
  plan_tier,
  total_minutes,
  avg_rating,
  churn_label
FROM `mgmt467-471119.netflix.churn_features_final`;

Executing query with job ID: ca05f30f-c18c-408a-8aa3-3dd3731ca697
Query executing: 0.39s


ERROR:
 404 Not found: Table mgmt467-471119:netflix.churn_features_final was not found in location US; reason: notFound, message: Not found: Table mgmt467-471119:netflix.churn_features_final was not found in location US

Location: US
Job ID: ca05f30f-c18c-408a-8aa3-3dd3731ca697




## Task 3: Evaluate Model

**🎯 Goal:** Evaluate the logistic regression model.  
**📌 Requirements:** Use ML.EVALUATE.

---

### 🧠 Prompt Template  
> Write a query to evaluate my logistic regression model using ML.EVALUATE.

---

### 👩‍🏫 Example Prompt  
> Evaluate the churn_model using ML.EVALUATE to get accuracy, precision, recall.

---

### ✅ Expected SQL Output
```sql
SELECT * FROM ML.EVALUATE(MODEL `your_dataset.churn_model`);
```

---

### 🔍 Checkpoint  
View performance metrics: accuracy, log_loss, precision, recall.


In [ ]:
%%bigquery --project $project_id
SELECT *
FROM ML.EVALUATE(MODEL `mgmt467-471119.netflix.churn_model_final`);

Executing query with job ID: bdf1cb04-171e-4c4c-a669-eb75879acfe9
Query executing: 0.32s


ERROR:
 404 Not found: Model mgmt467-471119:netflix.churn_model_final; reason: notFound, message: Not found: Model mgmt467-471119:netflix.churn_model_final

Location: US
Job ID: bdf1cb04-171e-4c4c-a669-eb75879acfe9




## Task 4: Predict Churn

**🎯 Goal:** Use ML.PREDICT to generate churn predictions.  
**📌 Requirements:** Apply model to same input table.

---

### 🧠 Prompt Template  
> Generate SQL to use ML.PREDICT on churn_model and return predictions by user_id.

---

### 👩‍🏫 Example Prompt  
> Predict churn using churn_model. Include user_id, predicted_churn_label, and prediction probability.

---

### ✅ Expected SQL Output
```sql
SELECT user_id, predicted_churn_label, predicted_churn_label_probs
FROM ML.PREDICT(MODEL `your_dataset.churn_model`,
      (SELECT * FROM `your_dataset.churn_features`));
```

---

### 🔍 Checkpoint  
Inspect top churn risk users. Validate probabilities.


In [ ]:
%%bigquery --project $project_id
SELECT
  user_id,
  predicted_churn_label,
  predicted_churn_label_probs
FROM ML.PREDICT(
  MODEL `mgmt467-471119.netflix.churn_model_final`,
  (
    SELECT *
    FROM `mgmt467-471119.netflix.churn_features_final`
  )
);

Executing query with job ID: 9b3a19cc-2ece-4559-bd07-dfc87620889d
Query executing: 0.34s


ERROR:
 404 Not found: Model mgmt467-471119:netflix.churn_model_final; reason: notFound, message: Not found: Model mgmt467-471119:netflix.churn_model_final

Location: US
Job ID: 9b3a19cc-2ece-4559-bd07-dfc87620889d

